In [1]:
import pandas as pd
from math import ceil, floor
import matplotlib.pyplot as plt
import seaborn as sns
import json
import matplotlib
from matplotlib import ticker
import pickle
%matplotlib inline

In [2]:
data = pd.read_csv("../components/artifacts/append_topics.csv", sep = "\t", index_col = 0)


/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data.head()

,Date,Page Type,Account Type,Author,Full Name,Full Text,Gender,Hashtags,Impact,Impressions,...,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10
0,2017-12-06 23:59:59,twitter,NaN,William82861606,William82861606 (William),funder conyers resigned franken resigning tomo...,male,"#trumpsexprobe, #metoo",0.0,282.0,...,0.000000,0.996263,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1,2017-12-06 23:59:58,twitter,NaN,Artis_Elemental,Artis_Elemental (ART...Is),alyssa_milano list sexual misconduct allegatio...,male,#metoo,24.5,5578.0,...,0.000000,0.990108,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000
2,2017-12-06 23:59:57,twitter,individual,lanah03,lanah03 (Laura Hopkins),deepinthehills sobbing silent longer,female,#metoo,5.4,2713.0,...,0.596061,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.384160,0.000000
3,2017-12-06 23:59:55,twitter,individual,lidskawasaki,lidskawasaki (dearlordbabyjesus),alyssa_milano time asking franken resign senat...,unknown,"#timepersonoftheyear, #metoo",0.0,330.0,...,0.000000,0.000000,0.0,0.0,0.000000,0.415957,0.0,0.0,0.125327,0.455225
4,2017-12-06 23:59:54,twitter,NaN,IAmMeliLopez,IAmMeliLopez (Melissa Lopez),nycchr public hearing begun stories deserve he...,female,"#sexualharassment, #metoo",0.0,13.0,...,0.000000,0.000000,0.0,0.0,0.991077,0.000000,0.0,0.0,0.000000,0.000000


In [4]:
data.columns

Index(['Date', 'Page Type', 'Account Type', 'Author', 'Full Name', 'Full Text',
       'Gender', 'Hashtags', 'Impact', 'Impressions', 'Thread Entry Type',
       'Thread Author', 'Twitter Followers', 'Twitter Following',
       'Twitter Tweets', 'Twitter Reply Count', 'Twitter Verified',
       'Twitter Retweets', 'Reach (new)', 'Region', 'topic_1', 'topic_2',
       'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8',
       'topic_9', 'topic_10'],
      dtype='object')

In [5]:
topic_score_cols = [x for x in data.columns if 'topic' in x]
topic_data = data[['Date','Author','Page Type','Account Type','Full Text'] + topic_score_cols]
topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)

# topic_data.head()

<ipython-input-5-c9b769e530c6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topic_data['Topic'] = topic_data[topic_score_cols].idxmax(axis=1)


In [17]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import pickle
import gensim
from gensim.models.ldamulticore import LdaMulticore
from gensim import corpora
from gensim.utils import simple_preprocess
import pandas as pd

pyLDAvis.enable_notebook()

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
data["Full Text"] = data["Full Text"].fillna("")
data = data[data["Thread Entry Type"] != "share"]

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
with open("../components/artifacts/dictionary", "rb") as f:
    dictionary = pickle.load(f)
tokenized_data = [simple_preprocess(text) for text in data["Full Text"] if len(text) >= 1]
corpus = [dictionary.doc2bow(doc, allow_update = False) for doc in tokenized_data]

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
def create_ldavis(model_path, corpus, dictionary):
    model = LdaMulticore.load(model_path)
    vis = gensimvis.prepare(model, corpus, dictionary)
    
    return vis, model

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
vis, model = create_ldavis("../components/artifacts/lda_model_num_topics=10-alpha=0.01-eta=0.21000000000000002", corpus, dictionary)

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
topic_words = model.show_topics(10, num_words=15, formatted=False)
for t_w in topic_words:
    print(t_w)
    print('-'*30)

(0, [('women', 0.014997833), ('movement', 0.009804509), ('timesup', 0.009371291), ('sexual', 0.008826868), ('harassment', 0.0052824127), ('time', 0.00438326), ('trump', 0.004208691), ('like', 0.0035648192), ('white', 0.0035333945), ('think', 0.0034105852), ('dont', 0.0033111963), ('don', 0.0030383118), ('black', 0.0028402072), ('era', 0.0026975197), ('hollywood', 0.0025533475)])
------------------------------
(1, [('sexual', 0.021513663), ('movement', 0.014517025), ('trump', 0.012803935), ('harassment', 0.01176969), ('assault', 0.01065527), ('women', 0.009963751), ('bill', 0.0066133332), ('clinton', 0.0061862622), ('realdonaldtrump', 0.0048469407), ('new', 0.0043290523), ('era', 0.0038327074), ('timesup', 0.003754294), ('men', 0.0036805975), ('power', 0.0035854876), ('donald', 0.0032392761)])
------------------------------
(2, [('latest', 0.026144277), ('daily', 0.018410696), ('thanks', 0.017632343), ('women', 0.012339301), ('movement', 0.011059779), ('new', 0.007979427), ('weinstein',

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
gensimvis.prepare(model, corpus, dictionary, mds = 'pcoa')

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.072510  0.056893       1        1  14.997496
5     -0.030369  0.017114       2        1  13.192072
8     -0.062609  0.049241       3        1  11.179872
4     -0.009064 -0.070103       4        1  11.004527
1      0.034140 -0.021630       5        1   9.801169
9     -0.007013  0.053928       6        1   9.164123
6      0.150779  0.050490       7        1   9.110437
2     -0.003557 -0.099644       8        1   7.596857
0      0.010723 -0.006581       9        1   7.168701
7     -0.010520 -0.029707      10        1   6.784744, topic_info=         Term           Freq          Total Category  logprob  loglift
98     latest   68258.000000   68258.000000  Default  30.0000  30.0000
96      daily   44798.000000   44798.000000  Default  29.0000  29.0000
38     thanks   49183.000000   49183.000000  Default  28.0000  28.0000
77   movement  317224.000000  317224.000000  Default  27.0000  27.0000
137    sexual  238057.000000  238057.000000  Default  26.0000  26.0000
..        ...            ...            ...      ...      ...      ...
94       time    5527.759523   98128.946451  Topic10  -5.7801  -0.1860
195       men    5835.724883  140259.438431  Topic10  -5.7259  -0.4890
33       like    5419.601340  128634.758915  Topic10  -5.7999  -0.4765
257    people    5271.559567   96957.827693  Topic10  -5.8276  -0.2215
77   movement    5114.893144  317224.052333  Topic10  -5.8577  -1.4370

[1105 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
17517      1  0.005665  aarp
17517      2  0.110912  aarp
17517      3  0.000508  aarp
17517      4  0.014018  aarp
17517      5  0.046340  aarp
...      ...       ...   ...
1934      10  0.020690    yr
42302      1  0.006197  zane
42302      2  0.039248  zane
42302      6  0.006197  zane
42302      8  0.944014  zane

[5440 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 9, 5, 2, 10, 7, 3, 1, 8])

In [14]:
vis

/shared/centos7/anaconda3/2021.05/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.072510  0.056893       1        1  14.998585
5     -0.030369  0.017114       2        1  13.187795
8     -0.062609  0.049241       3        1  11.176863
4     -0.009064 -0.070103       4        1  11.005797
1      0.034140 -0.021630       5        1   9.800434
9     -0.007013  0.053928       6        1   9.166603
6      0.150779  0.050490       7        1   9.110274
2     -0.003557 -0.099644       8        1   7.597768
0      0.010723 -0.006581       9        1   7.167371
7     -0.010520 -0.029707      10        1   6.788510, topic_info=         Term           Freq          Total Category  logprob  loglift
98     latest   68265.000000   68265.000000  Default  30.0000  30.0000
96      daily   44802.000000   44802.000000  Default  29.0000  29.0000
38     thanks   49187.000000   49187.000000  Default  28.0000  28.0000
77   movement  317223.000000  317223.000000  Default  27.0000  27.0000
137    sexual  238053.000000  238053.000000  Default  26.0000  26.0000
..        ...            ...            ...      ...      ...      ...
94       time    5530.827824   98122.858922  Topic10  -5.7801  -0.1859
195       men    5838.964128  140254.549599  Topic10  -5.7259  -0.4890
33       like    5422.609606  128626.299346  Topic10  -5.7999  -0.4764
257    people    5274.485659   96952.816341  Topic10  -5.8276  -0.2214
77   movement    5117.732275  317223.187843  Topic10  -5.8577  -1.4370

[1105 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
17517      1  0.005666  aarp
17517      2  0.110916  aarp
17517      3  0.000508  aarp
17517      4  0.014019  aarp
17517      5  0.046342  aarp
...      ...       ...   ...
1934      10  0.020686    yr
42302      1  0.006196  zane
42302      2  0.039244  zane
42302      6  0.006196  zane
42302      8  0.943917  zane

[5440 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 9, 5, 2, 10, 7, 3, 1, 8])

In [ ]:
print('\n\n'.join(topic_data[topic_data['Topic'] == 'Topic 4']['Full Text'].sample(30).tolist()))


In [1]:
coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=model, texts= corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

NameError: name 'gensim' is not defined

In [2]:

for topic in topic_score_cols:
    print(f'TOPIC {topic}')
    print('\n\n'.join(topic_data[topic_data['Topic'] == topic]['Full Text'].sample(30).tolist()))
    print('-'*30)

NameError: name 'topic_score_cols' is not defined

In [3]:
import os 
os.getcwd()


'/scratch/sridhar.p/metoo_v2/metoo_research/notebooks'